# Building a recommender for music artists

It's not possible to imagine modern online plattforms and services like Zalando, Amazon, Nextflix and Spotify without recommender systems. Especially for the latter businesses the performance of their recommendation systems is crucial to their success. At ComSysto, every employee has a generous amount of lab days, where one can work on own projects in order to test new technologies or to realize a cool idea. Since their great relevance makes recommenders very interesting, I decided to use some of my lab days to build and deploy such a system.

In this two-part blog-series you will see how to build a recommender model for music artists and how to deploy it with a REST API, so you can use it as a service from external applications. You can find the final result [here](https://denmei.github.io/). This first post is about building the model with the help of public user data from the musicplattform LastFm. Before I start with the fancy data-science stuff, I'd like to give a brief introduction to collaborative filtering, a central technique when it comes to recommendation systems.

## Brief Introduction to Collaborative Filtering
The basic idea behind collaborative filtering is that usually you get the best recommendations from people with a taste similar to yours. In general, two types of collaborative filtering can be distingiushed:
* **User-based**: Uses ratings from like-minded users to give a recommendation for the active user
* **Item-based**: Uses ratings from users who bought/liked the same items to give a recommendation for the active user

Since I am about to build an item-based collaborative filtering model in this post, let's dig deeper into this technique. The recommender is built by looking at users who have rated the same item, like for example a music artist. A rating does not necessarily have to be on a scale from 1 to 5 like on Amazon, instead the number of times a user listened to songs from a particular artist could also be used. The similarity between two users is calculated as sum of the distances between the ratings for every song the rated (compare the next graphic below).

![Item-based CF](img/icollaborative_filtering.png)

For calculating this distance, variuos metrics like the Euclidian Distance or the Cosine Similarity can be used. Once these similarities amongst users are available, they can be used to make recommendations for a specific user, using the following schema: *people who rate item X highly, like you, also tend to rate item Y highly, and you haven't rated item Y yet, so you should try it*.

## Preparation
First, we need to download the LastFM-datasets which can be found [here](http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/index.html) (I used the first, bigger one). Once you downloaded and opened the file, you can see that there are two datasets:
* **User Profiles**: The user profiles dataset contains data about 359.347 different users, including their id, geneder, age, country and the data when they signed up for the service.
![profiles](img/profiles.png)
* **User Data**: The user dataset contains 358.868 individual users' Last.fm-artist listening information. 
![plays](img/plays.png)

Now we are ready to start processing the data and building our model. Let's open a Jupyter Notebook (or something similar) in the same directory where the downloaded folder is located. I'll start with some imports first:

In [2]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import csr_matrix
import pickle
import warnings

pd.options.display.float_format = "{:.3f}".format

# ignore warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

Let's read the two datasets into Pandas DataFrames and do a little bit of cleaning. I'll drop the *artist_id*-column from the user-data since we will not need it. Furthermore, all rows where the artist's name is missing shall be removed, since we need this information for building the recommendation model. 

In [5]:
# read user profiles
user_profiles = pd.read_table("lastfm-dataset-360K/profiles.tsv", 
                         header=None, names=['user_id', 'gender', 'age', 'country', 'signup'])
print("Number of rows: " + str(len(user_profiles)))

# read user data
user_data = pd.read_table("lastfm-dataset-360K/plays.tsv", 
                         header=None, names=['user_id', 'artist_id', 'artist_name', 'plays'])

print("\nNumber of rows: " + str(len(user_data)))

# drop redundant column 
user_data.drop('artist_id', axis=1, inplace=True)

# drop all rows where we do not have an artist name
if user_data['artist_name'].isnull().sum() > 0:
    user_data = user_data.dropna(axis = 0, subset = ['artist_name'])

Number of rows: 359347
Number of unique users: 359347

Number of rows: 17535655
Number of users: 358868


### Reduce dataset size
The dataset is pretty big, which might cause some trouble when executing the subsequent steps on a normal computer. That's why I will reduce the size by filtering only for German user profiles and their corresponding data. By doing that, the number of considered profiles is reduced from 359.347 to 31.651 profiles.

In [6]:
german_profiles = user_profiles[user_profiles['country'] == 'Germany']
user_data_ger_profiles = german_profiles.merge(user_data, on="user_id", how='left')
print("Number of German profiles: " + str(len(german_profiles)))

Number of German profiles: 31651


### Reduce number of artists
Not so well known artists will have less plays from fewer users, which makes the data more noisy. This might have an effect on the recommender, since there might be a high sensitivity to instances where one individual user *loves* one less known artist.

To reduce this influence, we will filter only for the most popular users. 

Another advantage is that the file size will be reduced, leading to a better performance of the model when computing resources are limited.

First, let's create a table containing the total plays for each artist.

In [7]:
artist_plays = user_data_ger_profiles.groupby('artist_name')['plays'].sum().reset_index()
artist_plays.columns = ['artist_name', 'artist_total_plays']
print("Number of artists:" + str(len(artist_plays['artist_name'])))

     artist_name  artist_total_plays
0            !!!           14362.000
1  !action pact!              85.000
2          !cube              40.000
3       !deladap            1148.000
4       !distain             379.000
Number of artists:82816


With more than 83.000 artists, the probability that some artists have been played only a few times is high.

Let's find a threshold to define how many plays are needed to be a popular artist in the dataset by looking at the descriptives:

* The median artist only has round about 145 plays. 
* The the most popular artist has more than 2.9 million plays
* Only 1% of the artists has around 70.000 and more plays

To keep the dataset small, **I will choose a threshold of 90.000 total plays to define whether a artist is popular or not. This will reduce the number of artists to 645**.

In [10]:
# print statistics
print("Statistics: \n" + str(artist_plays['artist_total_plays'].describe()) + "\n")

# print quantiles for artist plays
print("Quantiles: \n" + str(artist_plays['artist_total_plays'].quantile(np.arange(.98, 1, .002))) + "\n")
threshold = 90000
popular_artist_plays = artist_plays[artist_plays['artist_total_plays'] > threshold]

print("Total number of artists: " + str(len(artist_plays['artist_name'])))
print("Number of artists with more plays than threshold: " + str(len(popular_artist_plays['artist_name'])))

Statistics: 
count     82816.000
mean       3649.069
std       32286.877
min           1.000
25%          36.000
50%         145.000
75%         618.000
max     2955844.000
Name: artist_total_plays, dtype: float64

Quantiles: 
0.980     29364.400
0.982     33647.000
0.984     39316.520
0.986     46431.380
0.988     56943.960
0.990     69368.700
0.992     87238.000
0.994    114542.490
0.996    165063.340
0.998    276339.520
1.000   2955844.000
Name: artist_total_plays, dtype: float64

Total number of artists: 82816
Number of artists with more plays than threshold: 645


Let's bring the artist and the profiles dataset together into one DataFrame. This will allow us to find out how often each user listened to a specific artist.

In [13]:
user_with_artist_plays = user_data_ger_profiles.merge(popular_artist_plays, on='artist_name', how='inner')
user_with_artist_plays = user_with_artist_plays.sort_values('artist_total_plays', ascending=False)
print(user_with_artist_plays.head(5))

                                       user_id gender    age  country  \
0     00000c289a1829a808ac09c00daf10bc3c4e223b      f 22.000  Germany   
4507  aba29c45c5067cba15e191da456a130ed84bcb14      f 26.000  Germany   
4479  aadcd8781ea372f3164b726ff10011d4ac73b9cc      m 32.000  Germany   
4478  aad472ed3b7ca0df1e4efc7c9b2436f52e221519      m 18.000  Germany   
4477  aad0cddae6587e92c7069b22d202adb99d53624e      f 15.000  Germany   

            signup artist_name    plays  artist_total_plays  
0      Feb 1, 2007   die Ärzte 1099.000         2955844.000  
4507   Sep 6, 2007   die Ärzte 2245.000         2955844.000  
4479  Nov 27, 2006   die Ärzte  802.000         2955844.000  
4478  Nov 24, 2006   die Ärzte  498.000         2955844.000  
4477  Oct 25, 2007   die Ärzte  648.000         2955844.000  


### Correct format
The objective of our recommender is to give k recommendations for a specific artist. That's why the model has to find the k artists that are most similar to a given musician, where similarity is defined by the distance between the artists. 

The k-nearest neighbor algorithm works perfectly well for this task. As a prerequesit, the data must be in a *mxn*-shaped matrix, where *n* is the number of artists and *m* is the number of users. So the format we need here is: ![collabfiltering](img/icollaborative_filtering.png)

Let's bring the data into this format with the following commands:

In [14]:
user_with_artist_plays = user_with_artist_plays.drop_duplicates(['user_id', 'artist_name'])
# pivot to create a dataframe with artists as rows and users as columns. 
# Fill with the number of plays per user and artist. Fill empty values with 0.
wide_artist_data = user_with_artist_plays.pivot(index = 'artist_name', columns = 'user_id', values = 'artist_total_plays').fillna(0)
# Transform to sparse matrix for more efficiency.
wide_artist_data_sparse = csr_matrix(wide_artist_data.values)

## Train model

The k-nearest neighbor algorithm computes the distance between a specific artist and the remaining artists in the dataset. The smaller the distance between two instances, the more similar they are. For each artist, the k best matches will be returned (the ones with the smallest distance).

![knn](img/knearestneighbor.jpeg)

There exists a large nummber of metrics you can choose to calculate this distance: Euclidian Distance, Pearson, Cosine, etc. I will use the cosine to calculate the distance between two items. The advantage of this metric is that you don't have to normalize the distances between the artists, which would be neccessary if we selected the euclidian distance or other metrics.

Let's train a NearestNeighbor-model on the dataset. 

In [15]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'auto')
model_knn.fit(wide_artist_data_sparse)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

## Make recommendations

We can use the model to make some recommendations.

Sklearn's kneighbors-model provides an operation called *kneighbors*, which we will use here. One has to hand over the index of the current artist's column as well as the required number of recommendations. The operation will return the distance and the index for each of the k recommendations. Since we are not interested in the indices but in the artists' names, we have to look the names up in the matrix. Let's give it a try with a random artist from our dataset:

In [16]:
query_index = np.random.choice(wide_artist_data.shape[0])
distances, indices = model_knn.kneighbors(wide_artist_data.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)

print(distances.flatten())
print("")
distance_len = len(distances.flatten())
for i in range(1, min(distance_len, distance_len + 1)):
    if i == 1:
        print("Artist: %s" % wide_artist_data.index[query_index])
    # it's necessary to use the recommended artists' indices for looking up their names in the dataset
    print("Recommendation %s: %s - %s " % (i, wide_artist_data.index[indices[0][i]], distances.flatten()[i]))

[1.11022302e-14 7.65758990e-01 7.96294190e-01 8.04213916e-01
 8.06852494e-01 8.14516981e-01]

Artist: kate nash
Recommendation 1: the kooks - 0.76575899024819 
Recommendation 2: feist - 0.7962941902590384 
Recommendation 3: arctic monkeys - 0.8042139155266868 
Recommendation 4: amy winehouse - 0.8068524936777223 
Recommendation 5: lily allen - 0.8145169805519052 


Since users will not be interested in recommendations for some random artist, the index of the artist one's looking for has to be looked up before calling the kneighbors-operation with this parameter. Let's check what recommendations we get if we like to hear songs by Bob Marley. From my own experience I'd say that the results are quite good for a simple recommender!

In [25]:
artist_name = "bob marley"
print("Artist: %s" % artist_name)
artists = user_with_artist_plays['artist_name'].unique()
artist_index = wide_artist_data.ix[artist_name].values.reshape(1, -1)
distances, indices = model_knn.kneighbors(artist_index, n_neighbors = 6)

distance_len = len(distances.flatten())
for i in range(1, min(distance_len, distance_len + 1)):
    print("Recommendation %s: %s - %s " % (i, wide_artist_data.index[indices[0][i]], distances.flatten()[i]))

Artist: bob marley
Recommendation 1: bob marley & the wailers - 0.7108467721143814 
Recommendation 2: gentleman - 0.7269421149045954 
Recommendation 3: seeed - 0.7633657191979925 
Recommendation 4: patrice - 0.769720806476048 
Recommendation 5: red hot chili peppers - 0.8101858833074674 


## Conclusion
In this blog post, I built a simple recommender that uses item-based collaborative filtering to recommend music artists. The next step is to make this model available to external applications, so that the recommender can for example be used on websites. This will be done in the next part of this series, where I will use Django and the Django Rest Framework to deploy the recommender.

## Save model
Since I want to deploy the model online, I will serialize and save it locally.

In [64]:
pickle.dump(model_knn, open('nn_recommender.sav', 'wb'))

## Save available artist-names
For my online artist-recommender, I also need the artist-name list to query for artists.

In [43]:
artists = pd.DataFrame(artists)
artists.to_csv("artists.csv")

## Deployment
I deployed the recommender in a simple **django-application on [Heroku](https://www.heroku.com/)**. 
The recommender can be used via a REST-API ([this website](https://denmei.github.io/) uses the API). 

Give it a try with [this](https://www.codepunker.com/tools/http-requests) http-request-service by making a POST-request (should be GET, I know):

* **URL**: https://ml-server-dm.herokuapp.com/music_recommender/api/artist_recommendation
* **Parameters**: 
    * artist: the name of the artist you are interested in
    * number: the number of recommendations you want to retrieve
